In [77]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np

from prettytable import PrettyTable

import glob
sns.set(style='darkgrid', palette = 'pastel')
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)

### Load agency data

In [14]:
file_list = glob.glob(r'agency\*.csv')

In [23]:
# combine 56 states data into one dataframe
df_agency = pd.DataFrame()
for file in file_list:
    df = pd.read_csv(file)
    df_agency = df.append(df_agency, ignore_index=True)

In [24]:
df_agency.shape

(18887, 13)

In [25]:
df_agency.head()

,ori,agency_name,agency_id,state_abbr,division_name,region_name,region_desc,county_name,agency_type_name,nibrs,nibrs_start_date,latitude,longitude
0,WY0010000,Albany County Sheriff's Office,21857,WY,Mountain,West,Region IV,ALBANY,County,True,2021-01-01T00:00:00.000Z,41.31,-105.59
1,WY0010100,Laramie Police Department,21858,WY,Mountain,West,Region IV,ALBANY,City,True,2018-01-01T00:00:00.000Z,41.67,-105.72
2,WY0010200,University of Wyoming,21859,WY,Mountain,West,Region IV,ALBANY,University or College,True,2021-01-01T00:00:00.000Z,41.67,-105.72
3,WY0020000,Big Horn County Sheriff's Office,21860,WY,Mountain,West,Region IV,BIG HORN,County,False,NaN,44.53,-107.99
4,WY0020100,Basin Police Department,21861,WY,Mountain,West,Region IV,BIG HORN,City,False,NaN,44.53,-107.99


print out the unique value for each feature (<20)

In [57]:
table = PrettyTable(max_width=10, align='l')

table.field_names = ['Feature', 'Data Type', '# of unique data #', 'unique value']
for column in df_agency.columns:
    data_type = str(df_agency[column].dtype)
    number_unique = len(df_agency[column].unique())
    if number_unique <20:
        unique_value = df_agency[column].value_counts().index.to_list()
    else:
        unique_value = "NA"
    table.add_row([column, data_type, number_unique, unique_value ])
print(table)

+------------------+-----------+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Feature          | Data Type | # of unique data # | unique value                                                                                                                                                                            |
+------------------+-----------+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| ori              | object    | 18887              | NA                                                                                                                                                                                      |
| agency_name      | object    | 14109  

### Load population data

data source: Intercensal Estimates of the Total Resident Population of States, by Population Distribution Branch
US Bureau of the Census

population data for 1960-2000 is in txt format; 2000-2023 data is in csv format.

#### 1. 6070 data

In [207]:
df = pd.read_table(r'population/st6070ts.txt', skiprows=12, header=None)
df.head()

,0
0,"April 1, July 1, July 1, July 1, July 1, July 1,"
1,1960 1960 1961 1962 1963 1964
2,"U.S. 179,323 179,975 182,973 185,738 188,438 191,085"
3,"Northeast 44,678 44,802 45,384 45,833 46,402 46,953"
4,"Midwest 51,619 51,715 52,185 52,583 53,073 53,655"


In [208]:
# split column 0 by whitespace
df['population'] = df.loc[:,0].apply(lambda x: ' '.join(x.split()))
df = df['population'].str.split(' ', expand=True)

In [209]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,April,"1,",July,"1,",July,"1,",July,"1,",July,"1,",July,"1,"
1,1960,1960,1961,1962,1963,1964,None,None,None,None,None,None
2,U.S.,"179,323","179,975","182,973","185,738","188,438","191,085",None,None,None,None,None
3,Northeast,"44,678","44,802","45,384","45,833","46,402","46,953",None,None,None,None,None
4,Midwest,"51,619","51,715","52,185","52,583","53,073","53,655",None,None,None,None,None


In [210]:
# find the row number which indicates the 2nd session of data
df[df.iloc[:,0].str.contains('1965')] 

,0,1,2,3,4,5,6,7,8,9,10,11
59,1965,1966,1967,1968,1969,1970,None,None,None,None,None,None


In [211]:
df_1 = df[1:58]
display(df_1.head(), df_1.tail())

df_2 = df[59:]
display(df_2.head(), df_2.tail())

,0,1,2,3,4,5,6,7,8,9,10,11
1,1960,1960,1961,1962,1963,1964,None,None,None,None,None,None
2,U.S.,"179,323","179,975","182,973","185,738","188,438","191,085",None,None,None,None,None
3,Northeast,"44,678","44,802","45,384","45,833","46,402","46,953",None,None,None,None,None
4,Midwest,"51,619","51,715","52,185","52,583","53,073","53,655",None,None,None,None,None
5,South,"54,973","55,174","56,158","57,179","58,008","58,853",None,None,None,None,None


,0,1,2,3,4,5,6,7,8,9,10,11
53,VA,"3,967","3,986","4,095","4,180","4,276","4,357",None,None,None,None,None
54,WA,"2,853","2,855","2,882","2,942","2,955","2,961",None,None,None,None,None
55,WV,"1,860","1,853","1,828","1,809","1,796","1,797",None,None,None,None,None
56,WI,"3,952","3,962","4,009","4,049","4,112","4,165",None,None,None,None,None
57,WY,330,331,337,333,336,339,None,None,None,None,None


,0,1,2,3,4,5,6,7,8,9,10,11
59,1965,1966,1967,1968,1969,1970,None,None,None,None,None,None
60,U.S.,"193,460","195,501","197,374","199,312","201,306","203,185",None,None,None,None,None
61,Northeast,"47,451","47,788","48,106","48,435","48,678","49,000",None,None,None,None,None
62,Midwest,"54,225","54,840","55,289","55,692","56,106","56,577",None,None,None,None,None
63,South,"59,579","60,205","60,771","61,489","62,190","62,798",None,None,None,None,None


,0,1,2,3,4,5,6,7,8,9,10,11
111,VA,"4,411","4,456","4,508","4,558","4,614","4,648",None,None,None,None,None
112,WA,"2,967","3,057","3,174","3,270","3,343","3,409",None,None,None,None,None
113,WV,"1,786","1,775","1,769","1,763","1,746","1,744",None,None,None,None,None
114,WI,"4,232","4,274","4,303","4,345","4,378","4,418",None,None,None,None,None
115,WY,332,323,322,324,329,332,None,None,None,None,None


In [212]:
# fix column name
df_1 = df_1.rename(columns={1: 1960_0, 2:1960,3:1961,4:1962, 5:1963, 6:1964})

In [213]:
# pick first 7 columns
df_1 = df_1.iloc[1:, 0:7].reset_index(drop=True)
df_1.head()

,0,19600,1960,1961,1962,1963,1964
0,U.S.,"179,323","179,975","182,973","185,738","188,438","191,085"
1,Northeast,"44,678","44,802","45,384","45,833","46,402","46,953"
2,Midwest,"51,619","51,715","52,185","52,583","53,073","53,655"
3,South,"54,973","55,174","56,158","57,179","58,008","58,853"
4,West,"28,053","28,284","29,246","30,142","30,955","31,624"


In [214]:
# fix column name
df_2 = df_2.rename(columns={1: 1965, 2:1966,3:1967,4:1968, 5:1969, 6:1970})
df_2 = df_2.iloc[1:, 0:7].reset_index(drop=True)
df_2.head()

,0,1965,1966,1967,1968,1969,1970
0,U.S.,"193,460","195,501","197,374","199,312","201,306","203,185"
1,Northeast,"47,451","47,788","48,106","48,435","48,678","49,000"
2,Midwest,"54,225","54,840","55,289","55,692","56,106","56,577"
3,South,"59,579","60,205","60,771","61,489","62,190","62,798"
4,West,"32,205","32,667","33,207","33,696","34,332","34,809"


In [217]:
# combine df_1 and df_2 to df_60_70, delete the 19600 column

df_60_70 = pd.merge(df_1,df_2, on=0)
df_60_70 = df_60_70.drop(columns=19600)

In [257]:
# change data type to int

df_60_70.replace(',','', regex=True, inplace=True)
cols = df_60_70.columns.to_list()[1:]
df_60_70[cols] = df_60_70[cols].applymap(np.int64)

In [264]:
df_60_70 = df_60_70.T

In [266]:
# set the first row as the header
new_header = df_60_70.iloc[0] #grab the first row for the header
df_60_70 = df_60_70[1:] #take the data less the header row
df_60_70.columns = new_header #set the header row as the df header

In [273]:
df_60_70.columns

Index(['U.S.', 'Northeast', 'Midwest', 'South', 'West', 'AL', 'AK', 'AZ', 'AR',
       'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
       'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE',
       'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI',
       'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'],
      dtype='object', name=0)

In [274]:
df_60_70.drop(columns=['Northeast', 'Midwest', 'South', 'West'], inplace=True)

C:\Users\guym\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [278]:
# the numbers are in thousands
df_60_70 = df_60_70*1000

In [279]:
df_60_70

,U.S.,AL,AK,AZ,AR,CA,CO,CT,DE,DC,FL,GA,HI,ID,IL,IN,IA,KS,KY,LA,ME,MD,MA,MI,MN,...,MT,NE,NV,NH,NJ,NM,NY,NC,ND,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
1960,179975000,3274000,229000,1321000,1789000,15870000,1769000,2544000,449000,765000,5004000,3956000,642000,671000,10086000,4674000,2756000,2183000,3041000,3260000,975000,3113000,5160000,7834000,3425000,...,679000,1417000,291000,609000,6103000,954000,16838000,4573000,634000,9734000,2336000,1772000,11329000,855000,2392000,683000,3575000,9624000,900000,389000,3986000,2855000,1853000,3962000,331000
1961,182973000,3316000,238000,1407000,1806000,16497000,1844000,2586000,461000,778000,5243000,4015000,659000,684000,10130000,4730000,2756000,2215000,3054000,3287000,995000,3176000,5219000,7893000,3470000,...,696000,1446000,315000,618000,6265000,965000,17061000,4663000,641000,9854000,2380000,1787000,11392000,858000,2409000,693000,3622000,9820000,936000,390000,4095000,2882000,1828000,4009000,337000
1962,185738000,3323000,246000,1471000,1853000,17072000,1899000,2647000,469000,788000,5458000,4086000,684000,692000,10280000,4736000,2750000,2231000,3079000,3345000,994000,3263000,5263000,7933000,3513000,...,698000,1464000,352000,632000,6376000,979000,17301000,4707000,637000,9929000,2427000,1818000,11355000,871000,2423000,705000,3673000,10053000,958000,393000,4180000,2942000,1809000,4049000,333000
1963,188438000,3358000,256000,1521000,1875000,17668000,1936000,2727000,483000,798000,5628000,4172000,682000,683000,10402000,4799000,2747000,2217000,3096000,3377000,993000,3386000,5344000,8058000,3531000,...,703000,1476000,397000,649000,6531000,989000,17461000,4742000,644000,9986000,2439000,1853000,11424000,876000,2460000,708000,3718000,10159000,974000,397000,4276000,2955000,1796000,4112000,336000
1964,191085000,3395000,263000,1556000,1897000,18151000,1970000,2798000,497000,798000,5781000,4258000,700000,680000,10580000,4856000,2746000,2209000,3129000,3446000,993000,3492000,5448000,8187000,3558000,...,706000,1482000,426000,663000,6660000,1006000,17589000,4802000,649000,10080000,2446000,1888000,11519000,885000,2475000,701000,3771000,10270000,978000,399000,4357000,2961000,1797000,4165000,339000
1965,193460000,3443000,271000,1584000,1894000,18585000,1985000,2857000,507000,797000,5954000,4332000,704000,686000,10693000,4922000,2742000,2206000,3140000,3496000,997000,3600000,5502000,8357000,3592000,...,706000,1471000,444000,676000,6767000,1012000,17734000,4863000,649000,10201000,2440000,1937000,11620000,893000,2494000,692000,3798000,10378000,994000,404000,4411000,2967000,1786000,4232000,332000
1966,195501000,3464000,271000,1614000,1899000,18858000,2007000,2903000,516000,791000,6104000,4379000,710000,689000,10836000,4999000,2762000,2200000,3147000,3550000,999000,3695000,5535000,8512000,3617000,...,707000,1456000,446000,681000,6851000,1007000,17843000,4896000,647000,10330000,2454000,1969000,11664000,899000,2520000,683000,3822000,10492000,1009000,413000,4456000,3057000,1775000,4274000,323000
1967,197374000,3458000,278000,1646000,1901000,19176000,2053000,2935000,525000,791000,6242000,4408000,723000,688000,10947000,5053000,2793000,2197000,3172000,3581000,1004000,3757000,5594000,8630000,3659000,...,701000,1457000,449000,697000,6928000,1000000,17935000,4952000,626000,10414000,2489000,1979000,11681000,909000,2533000,671000,3859000,10599000,1019000,423000,4508000,3174000,1769000,4303000,322000
1968,199312000,3446000,285000,1682000,1902000,19394000,2120000,2964000,534000,778000,6433000,4482000,734000,695000,10995000,5093000,2803000,2216000,3195000,3603000,994000,3815000,5618000,8696000,3703000,...,700000,1467000,464000,709000,7005000,994000,18051000,5004000,621000,10516000,2503000,2004000,11741000,922000,2559000,669000,3878000,10819000,1029000,430000,4558000,3270000,1763000,4345000,324000
1969,201306000,3440000,296000,1737000,1913000,19711000,2166000,3000000,540000,762000,6641000,4551000,750000,707000,11039000,5143000,2805000,2236000,3198000,3619000,992000,3868000,5650000,8781000,3758000,...,694000,1474000,480000,724

#### 2. 7080 data

In [293]:
df = pd.read_table(r'population/st7080ts.txt', skiprows=12, header=None)
df.head()

,0
0,Fip St 4/1/70 7/71 7/72 7/73 7/74 7/75
1,1 AL 3444354 3497076 3539400 3579780 3626499 3678814
2,2 AK 302583 315510 324464 330543 341063 376170
3,4 AZ 1775399 1895814 2008291 2124438 2223196 2284847
4,5 AR 1923322 1972312 2018638 2059256 2101403 2159526


In [294]:
# split column 0 by whitespace
df['population'] = df.loc[:,0].apply(lambda x: ' '.join(x.split()))
df = df['population'].str.split(' ', expand=True)

In [295]:
# find the row number which indicates the start of the 2nd session of data
df[df.iloc[:,0].str.contains('7/76')] 

,0,1,2,3,4,5,6,7
53,7/76,7/77,7/78,7/79,4/1/80,None,None,None


In [296]:
df_1 = df[0:53]
display(df_1.head(), df_1.tail())

df_2 = df[53:106]
display(df_2.head(), df_2.tail())

,0,1,2,3,4,5,6,7
0,Fip,St,4/1/70,7/71,7/72,7/73,7/74,7/75
1,1,AL,3444354,3497076,3539400,3579780,3626499,3678814
2,2,AK,302583,315510,324464,330543,341063,376170
3,4,AZ,1775399,1895814,2008291,2124438,2223196,2284847
4,5,AR,1923322,1972312,2018638,2059256,2101403,2159526


,0,1,2,3,4,5,6,7
48,53,WA,3413244,3447553,3447885,3478782,3549946,3621339
49,54,WV,1744237,1770657,1797648,1806360,1815447,1842250
50,55,WI,4417821,4462155,4502412,4524244,4545782,4578986
51,56,WY,332416,340285,347345,354061,365501,381695
52,US,203302031,206827026,209283905,211357481,213341554,215465255,None


,0,1,2,3,4,5,6,7
53,7/76,7/77,7/78,7/79,4/1/80,None,None,None
54,1,AL,3735139,3780403,3831836,3866248,3893888,None
55,2,AK,400969,403436,404766,402753,401851,None
56,4,AZ,2346157,2425197,2515316,2635571,2718215,None
57,5,AR,2170161,2209010,2243127,2271333,2286435,None


,0,1,2,3,4,5,6,7
101,53,WA,3693610,3776050,3889073,4017632,4132156,None
102,54,WV,1879503,1908088,1923395,1942146,1949644,None
103,55,WI,4595904,4626514,4646108,4682811,4705767,None
104,56,WY,396952,413354,432880,454378,469557,None
105,US,217562735,219759869,222095080,224567241,226545805,None,None


In [297]:
# fix column name
df_1 = df_1.rename(columns={2: 1970, 3:1971, 4:1972, 5:1973, 6:1974, 7:1975})
df_2 = df_2.rename(columns = {2:1976, 3:1977, 4:1978, 5:1979, 6: 1980})

In [298]:
df_1 = df_1.iloc[1:, 1:].reset_index(drop=True)
df_2 = df_2.iloc[1:, 1:].reset_index(drop=True)

In [300]:
# shift the last row to right
df_1.iloc[51,:] = df_1.iloc[51,:].shift(periods=1, axis='rows', fill_value='U.S.')
df_2.iloc[51,:] = df_2.iloc[51,:].shift(periods=1, axis='rows', fill_value='U.S.')

In [308]:
# drop the last column 
df_2.drop(columns=7, inplace=True)

In [311]:
# combine df_1 and df_2 to df_70_80
df_70_80 = pd.merge(df_1,df_2, on=1)

In [320]:
df_70_80 = df_70_80.set_index(1).T

In [322]:
# shift the last column to first
first_column = df_70_80.pop('U.S.')
df_70_80.insert(0, 'U.S.', first_column) 

In [380]:
# change data type to int
df_70_80 = df_70_80.astype(int)

#### 3. 8090 data

In [340]:
df = pd.read_table(r'population/st8090ts.txt', skiprows=9, header=None)
df.head()

,0
0,4/80cen 7/81 7/82 7/83 7/84
1,US 226545805 229465714 231664458 233791994 235824902
2,AL 3893888 3918531 3925266 3934102 3951820
3,AK 401851 418491 449606 488417 513702
4,AZ 2718215 2810107 2889861 2968925 3067135


In [341]:
# split column 0 by whitespace
df['population'] = df.loc[:,0].apply(lambda x: ' '.join(x.split()))
df = df['population'].str.split(' ', expand=True)

In [342]:
# find the row number which indicates the start of the 2nd session of data
df[df.iloc[:,0].str.contains('7/85')] 

,0,1,2,3,4,5,6
53,7/85,7/86,7/87,7/88,7/89,4/90cen,None


In [343]:
df_1 = df[0:53]
display(df_1.head(), df_1.tail())

df_2 = df[53:]
display(df_2.head(), df_2.tail())

,0,1,2,3,4,5,6
0,4/80cen,7/81,7/82,7/83,7/84,None,None
1,US,226545805,229465714,231664458,233791994,235824902,None
2,AL,3893888,3918531,3925266,3934102,3951820,None
3,AK,401851,418491,449606,488417,513702,None
4,AZ,2718215,2810107,2889861,2968925,3067135,None


,0,1,2,3,4,5,6
48,VA,5346818,5444097,5492783,5564657,5643870,None
49,WA,4132156,4235731,4276552,4300266,4343656,None
50,WV,1949644,1954124,1949604,1945061,1927697,None
51,WI,4705767,4726343,4728870,4721438,4735563,None
52,WY,469557,491712,506400,510345,504896,None


,0,1,2,3,4,5,6
53,7/85,7/86,7/87,7/88,7/89,4/90cen,None
54,US,237923795,240132887,242288918,244498982,246819230,248709873
55,AL,3972523,3991569,4015264,4023844,4030222,4040587
56,AK,532495,544268,539309,541983,547159,550043
57,AZ,3183538,3308262,3437103,3535183,3622185,3665228


,0,1,2,3,4,5,6
101,VA,5715153,5811700,5932268,6036909,6120246,6187358
102,WA,4400098,4452720,4531901,4639893,4746316,4866692
103,WV,1906831,1882350,1857585,1830215,1806568,1793477
104,WI,4747767,4755618,4777919,4822388,4856574,4891769
105,WY,499695,495633,476965,465101,458374,453588


In [344]:
# fix column name
df_1 = df_1.rename(columns={1: '1980_cen', 2:1981, 3:1982, 4:1983, 5:1984})
df_2 = df_2.rename(columns = {1:1985, 2:1986, 3:1987, 4:1988, 5: 1989, 6: '1990_cen'})

In [345]:
df_1 = df_1.iloc[1:, :-1].reset_index(drop=True)
df_2 = df_2.iloc[1:, :].reset_index(drop=True)

In [348]:
# combine df_1 and df_2 to df_80_90
df_80_90 = pd.merge(df_1,df_2, on=0)

In [352]:
df_80_90 = df_80_90.set_index(0).T

In [355]:
# change first column name to U.S.
df_80_90.rename(columns={'US':'U.S.'}, inplace=True )

In [356]:
df_80_90.shape

(11, 52)

In [376]:
# change data type to int
df_80_90 = df_80_90.astype(int)

#### 4. 9000 data

data was preprocessed before using txt editor

In [386]:
df = pd.read_table(r'population/st9000ts.txt', skiprows=16, header=None)
df.head()

,0
0,Number Code 7/1/2000 7/1/1999 7/1/1998 7/1/1997 7/1/1996 7/1/1995 7/1/1994 7/1/1993 7/1/1992 7/1/1991 7/1/1990 4/1/1990
1,"1 00 275,264,999 272,787,485 270,334,431 267,846,741 265,300,335 262,876,514 260,401,091 257,857,622 255,104,027 252,208,537 249,470,539 248,790,925"
2,"1 01 4,385,739 4,369,224 4,350,594 4,320,692 4,290,678 4,263,125 4,233,214 4,193,639 4,139,880 4,091,397 4,048,526 4,040,389"
3,"1 02 622,091 619,596 615,292 608,867 604,924 601,349 600,628 596,999 587,105 569,297 553,110 550,043"
4,"1 04 4,882,966 4,774,104 4,666,824 4,552,210 4,432,497 4,307,149 4,147,754 3,993,996 3,868,031 3,762,955 3,679,118 3,665,339"


In [387]:
# split column 0 by whitespace
df['population'] = df.loc[:,0].apply(lambda x: ' '.join(x.split()))
df = df['population'].str.split(' ', expand=True)

In [434]:
# set the first row as header
df.columns = df.iloc[0]
df = df[1:]

##### map fips code with state code

In [429]:
df_fips = pd.read_table(r'population/fips.txt', sep=',', dtype='str')

In [430]:
# remove white space 
df_fips[' st']= df_fips[' st'].apply(lambda x: x.replace(' ', ''))
df_fips[' stusps']= df_fips[' stusps'].apply(lambda x: x.replace(' ', ''))

In [431]:
dict_df = dict(zip(df_fips[' st'], df_fips[' stusps']))
print(dict_df)

{'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY'}


In [432]:
# add 00, U.S. to the dict_df
dict_df['00'] ='U.S.'

In [437]:
df['Code'] = df['Code'].map(dict_df)

C:\Users\guym\AppData\Local\Temp/ipykernel_21340/2354372885.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Code'] = df['Code'].map(dict_df)


In [439]:
# set 'Code' column as index column 
df.set_index('Code', inplace=True)

In [442]:
df.drop(columns='Number', inplace=True)

C:\Users\guym\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [460]:
# fix data type
df = df.replace(',', '', regex=True)
df = df[:-1]
df = df.astype(int)

In [467]:
# reorder columns
df = df[df.columns[::-1]]

In [469]:
df_90_00  = df.T

In [474]:
df_90_00.shape

(12, 52)

#### 5. 0009 data

In [518]:
df = pd.read_csv(r'population\2000-2009.csv', header=3)

In [519]:
# remove rows 1-4 and rows 56-62
df.drop(index=[1,2,3,4,56,57,58,59,60,61,62], inplace=True)
# remove the last two columns
df = df.iloc[:, :-2]

In [520]:
# fix state name
df.iloc[:,0] = df.iloc[:,0].str.replace('.', '', regex=True)

In [521]:
dict_df2 = dict(zip(df_fips['stname'], df_fips[' stusps']))
dict_df2['United States'] = 'U.S.'
print(dict_df2)

{'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'United States': 'U.S.'}


In [522]:
df['Unnamed: 0'] = df['Unnamed: 0'].map(dict_df2)

In [525]:
# set state abbr. as index
df.set_index('Unnamed: 0', inplace=True)

In [526]:
# reorder columns
df = df[df.columns[::-1]]

# rename column names
name_list = np.arange(2000,2010,1)
df.columns = name_list

In [534]:
# change data type
df.replace(',', '', regex=True, inplace=True)

In [537]:
df = df.astype(int)

In [539]:
df_00_09 = df.T

In [541]:
df_00_09.shape

(10, 52)

#### 6. 1019 data

In [551]:
df = pd.read_excel(r'population\2010-2019.xlsx', header=3)

In [557]:
# drop 2nd and 3rd columns
df.drop(columns=['Census', 'Estimates Base'], inplace=True)

In [562]:
# drop row 1-4 and 56-62
df.drop(index=[1,2,3,4,56,57,58,59,60,61,62], inplace=True)

In [564]:
# fix state name
df.iloc[:,0] = df.iloc[:,0].str.replace('.', '', regex=True)
df['Unnamed: 0'] = df['Unnamed: 0'].map(dict_df2)

In [568]:
# set state abbr. as index
df.set_index('Unnamed: 0', inplace=True)

In [571]:
# change data type from float to int
df  = df.astype(int)

In [573]:
df_10_19 = df.T

In [574]:
df_10_19.shape

(10, 52)

#### 7. 2023 data

In [579]:
df = pd.read_excel(r'population\2020-2023.xlsx', header=3)

In [582]:
# drop first column
df.drop(columns='Unnamed: 1', inplace=True)

In [586]:
# drop row 1-4 and 56-63
df.drop(index=[1,2,3,4,56,57,58,59,60,61,62,63], inplace=True)

In [588]:
# fix state name
df.iloc[:,0] = df.iloc[:,0].str.replace('.', '', regex=True)
df['Unnamed: 0'] = df['Unnamed: 0'].map(dict_df2)

In [590]:
# set state abbr. as index
df.set_index('Unnamed: 0', inplace=True)

In [593]:
# change data type from float to int
df  = df.astype(int)

In [594]:
df_20_23 = df.T

In [595]:
df_20_23.shape

(4, 52)

#### Combine census data

In [599]:
df_60_23 = pd.concat([df_60_70, df_70_80, df_80_90,df_90_00, df_00_09, df_10_19, df_20_23])

In [614]:
df_60_23 = df_60_23.reset_index()

In [619]:
# delete row 10, 22,32,33, 44
df_60_23.drop(index=[10,22,32,33,44], inplace=True)

In [628]:
df_60_23.reset_index(drop=True, inplace=True )

In [634]:
df_60_23.head(50)

,index,U.S.,AL,AK,AZ,AR,CA,CO,CT,DE,DC,FL,GA,HI,ID,IL,IN,IA,KS,KY,LA,ME,MD,MA,MI,...,MT,NE,NV,NH,NJ,NM,NY,NC,ND,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,1960,179975000,3274000,229000,1321000,1789000,15870000,1769000,2544000,449000,765000,5004000,3956000,642000,671000,10086000,4674000,2756000,2183000,3041000,3260000,975000,3113000,5160000,7834000,...,679000,1417000,291000,609000,6103000,954000,16838000,4573000,634000,9734000,2336000,1772000,11329000,855000,2392000,683000,3575000,9624000,900000,389000,3986000,2855000,1853000,3962000,331000
1,1961,182973000,3316000,238000,1407000,1806000,16497000,1844000,2586000,461000,778000,5243000,4015000,659000,684000,10130000,4730000,2756000,2215000,3054000,3287000,995000,3176000,5219000,7893000,...,696000,1446000,315000,618000,6265000,965000,17061000,4663000,641000,9854000,2380000,1787000,11392000,858000,2409000,693000,3622000,9820000,936000,390000,4095000,2882000,1828000,4009000,337000
2,1962,185738000,3323000,246000,1471000,1853000,17072000,1899000,2647000,469000,788000,5458000,4086000,684000,692000,10280000,4736000,2750000,2231000,3079000,3345000,994000,3263000,5263000,7933000,...,698000,1464000,352000,632000,6376000,979000,17301000,4707000,637000,9929000,2427000,1818000,11355000,871000,2423000,705000,3673000,10053000,958000,393000,4180000,2942000,1809000,4049000,333000
3,1963,188438000,3358000,256000,1521000,1875000,17668000,1936000,2727000,483000,798000,5628000,4172000,682000,683000,10402000,4799000,2747000,2217000,3096000,3377000,993000,3386000,5344000,8058000,...,703000,1476000,397000,649000,6531000,989000,17461000,4742000,644000,9986000,2439000,1853000,11424000,876000,2460000,708000,3718000,10159000,974000,397000,4276000,2955000,1796000,4112000,336000
4,1964,191085000,3395000,263000,1556000,1897000,18151000,1970000,2798000,497000,798000,5781000,4258000,700000,680000,10580000,4856000,2746000,2209000,3129000,3446000,993000,3492000,5448000,8187000,...,706000,1482000,426000,663000,6660000,1006000,17589000,4802000,649000,10080000,2446000,1888000,11519000,885000,2475000,701000,3771000,10270000,978000,399000,4357000,2961000,1797000,4165000,339000
5,1965,193460000,3443000,271000,1584000,1894000,18585000,1985000,2857000,507000,797000,5954000,4332000,704000,686000,10693000,4922000,2742000,2206000,3140000,3496000,997000,3600000,5502000,8357000,...,706000,1471000,444000,676000,6767000,1012000,17734000,4863000,649000,10201000,2440000,1937000,11620000,893000,2494000,692000,3798000,10378000,994000,404000,4411000,2967000,1786000,4232000,332000
6,1966,195501000,3464000,271000,1614000,1899000,18858000,2007000,2903000,516000,791000,6104000,4379000,710000,689000,10836000,4999000,2762000,2200000,3147000,3550000,999000,3695000,5535000,8512000,...,707000,1456000,446000,681000,6851000,1007000,17843000,4896000,647000,10330000,2454000,1969000,11664000,899000,2520000,683000,3822000,10492000,1009000,413000,4456000,3057000,1775000,4274000,323000
7,1967,197374000,3458000,278000,1646000,1901000,19176000,2053000,2935000,525000,791000,6242000,4408000,723000,688000,10947000,5053000,2793000,2197000,3172000,3581000,1004000,3757000,5594000,8630000,...,701000,1457000,449000,697000,6928000,1000000,17935000,4952000,626000,10414000,2489000,1979000,11681000,909000,2533000,671000,3859000,10599000,1019000,423000,4508000,3174000,1769000,4303000,322000
8,1968,199312000,3446000,285000,1682000,1902000,19394000,2120000,2964000,534000,778000,6433000,4482000,734000,695000,10995000,5093000,2803000,2216000,3195000,3603000,994000,3815000,5618000,8696000,...,700000,1467000,464000,709000,7005000,994000,18051000,5004000,621000,10516000,2503000,2004000,11741000,922000,2559000,669000,3878000,10819000,1029000,430000,4558000,3270000,1763000,4345000,324000
9,1969,201306000,3440000,296000,1737000,1913000,19711000,2166000,3000000,540000,762000,6641000,4551000,750000,707000,11039000,5143000,2805000,2236000,3198000,3619000,992000,3868000,5650000,8781000,...,694000,1474000,480000,724000,7095000,1011000,18105000,5031000,621000,10563000,2535

In [633]:
# fix year for rows 30- 39
i = np.arange(30,40,1)
y = np.arange(1990,2020, 1)
for i, y in zip(i, y):
    df_60_23.iloc[i,0] = y

In [636]:
# fix column name 
df_60_23.rename(columns={'index':'year'}, inplace=True)

### save census data

In [638]:
df_60_23.to_csv('census_60_23.csv')